[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io)

# AI Program

## Machine Learning - UnSupervised Learning - Manifold Learning - UMAP

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 13/09/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0067ManifoldLearningIsoMap.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from umap import UMAP

# Miscellaneous
from platform import python_version
import random

# Typing
from typing import Callable, Dict, List, Optional, Self, Set, Tuple, Union
from numpy.typing import NDArray

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

In [ ]:
# Courses Packages


In [ ]:
# General Auxiliary Functions

def PlotMnistImages( mX: NDArray, vY: NDArray, numRows: int, /, *, numCols: Optional[int] = None, tuImgSize: Tuple = (28, 28), randomChoice: bool = True, lClasses: Optional[List] = None, hF: Optional[plt.Figure] = None ) -> plt.Figure:

    numSamples  = mX.shape[0]
    numPx       = mX.shape[1]

    if numCols is None:
        numCols = numRows

    tFigSize = (numCols * 3, numRows * 3)

    if hF is None:
        hF, vHa = plt.subplots(numRows, numCols, figsize = tFigSize)
    else:
        vHa = hF.axes
    
    vHa = np.atleast_1d(vHa) #<! To support numImg = 1
    vHa = vHa.flat
    
    for kk in range(numRows * numCols):
        idx = np.random.choice(numSamples) if randomChoice else kk
        mI  = np.reshape(mX[idx, :], tuImgSize)
    
        # hA[kk].imshow(mI.clip(0, 1), cmap = 'gray')
        if len(tuImgSize) == 2:
            vHa[kk].imshow(mI, cmap = 'gray')
        elif len(tuImgSize) == 3:
            vHa[kk].imshow(mI)
        else:
            raise ValueError(f'The length of the image size tuple is {len(tuImgSize)} which is not supported')
        vHa[kk].tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, 
                           labelleft = False, labeltop = False, labelright = False, labelbottom = False)
        if lClasses is None:
            vHa[kk].set_title(f'Index = {idx}, Label = {vY[idx]}')
        else:
            vHa[kk].set_title(f'Index = {idx}, Label = {lClasses[vY[idx]]}')
    
    return hF

def PlotScatterData( mX: NDArray, vL: Optional[NDArray] = None, lLabel: Optional[List[str]] = None, hA: Optional[plt.Axes] = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = ELM_SIZE_DEF, edgeColor: int = EDGE_COLOR ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    numSamples = mX.shape[0]

    if vL is None:
        vL = np.zeros(numSamples)
    
    if lLabel is None:
        lLabel = [str(ii) for ii in range(int(np.max(vL)) + 1)] 
    
    vU = np.unique(vL)
    numClusters = len(vU)

    for ii in range(numClusters):
        vIdx = vL == vU[ii]
        hA.scatter(mX[vIdx, 0], mX[vIdx, 1], s = markerSize, edgecolor = edgeColor, label = lLabel[ii])
    
    hA.set_xlabel('${{x}}_{{1}}$')
    hA.set_ylabel('${{x}}_{{2}}$')
    hA.grid()
    hA.legend()

    return hA

## Dimensionality Reduction

Given a data set $\mathcal{X} = \{ \boldsymbol{x}_{i} \in \mathbb{R}^{D} \}_{i = 1}$ _Dimensionality Reduction_ seeks for $\mathcal{Z} = \{ \boldsymbol{z}_{i} \in \mathbb{R}^{d} \}_{i = 1}$ where $d \leq D$ and some properties of the data are preserved.  
The _Dimensionality Reduction_ function maps each sample from the high dimensional space to a sample in the low dimensional space.  

Motivations for Dimensionality Reduction
 * Feature Engineering  
   Dimensionality reduction may assist algorithms "focus" on the important relations between samples.  
   Specifically, for High Dimensional Data it may reduce issues caused by "Curse of Dimensionality" and make the distance useful.  
   <font color='magenta'>Example</font>: A common synergy is _Dimensionality Reduction_ followed by _Clustering_.  
   <font color='magenta'>Example</font>: Using PCA to enhance the data SNR.
 * Visualization  
   High Dimensional data can not be visualized geometrically.  
   Yet such analysis is important in the _EDA_ and _Results Analysis_.  
   _Dimensionality Reduction_ to a 2D / 3D, with the assumption the important relations are well kept in the process, may assist with the analysis.
 * Run Time Optimization  
   Most algorithms have a run time complexity dependency on the dimension of the data.  
   Dimensionality reduction, by nature, will accelerate such cases.  
   <font color='magenta'>Example</font>: Using PCA to reduce 50% of the dimensions while preserving 95% of the energy.

Broadly, _Dimensionality Reduction_ is defined by 3 types:

 * Linear  
   The low dimension data is a linear / affine function of the high dimensional data: $\boldsymbol{z}_{i} = \boldsymbol{W} \boldsymbol{x}_{i} + \boldsymbol{b}$.  
   The matrix and bias vector are the solution of the algorithm.
   Methods such as [Principle Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA), [Independent Component Analysis](https://en.wikipedia.org/wiki/Independent_component_analysis) ICA, [Canonical Correlation Analysis](https://en.wikipedia.org/wiki/Canonical_correlation) (CCA).
 * Non Linear  
   The low dimension sample is a non linear function of the high dimension sample.  
   Methods such as [Kernel PCA](https://en.wikipedia.org/wiki/Kernel_principal_component_analysis), [MultiDimensional Scaling](https://en.wikipedia.org/wiki/Multidimensional_scaling).  
   Some Non Linear Methods are called _Manifold Learning_ which try to learn the local metric of the data: Laplacian Eigenmaps, [T-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) and UMAP.

<br>

* <font color='brown'>(**#**)</font> One may apply Dimensionality Reduction on labeled data.  
There are specialized algorithms for such case. A simple linear one is given by the [Linear Discriminant Analysis](https://en.wikipedia.org/wiki/Linear_discriminant_analysis) (LDA) algorithm.
* <font color='brown'>(**#**)</font> _Dimensionality Reduction_ is potentially the _Optimal Feature Engineering_.  
* <font color='brown'>(**#**)</font> _Feature Selection_ is also a type of _Dimensionality Reduction_.
* <font color='brown'>(**#**)</font> One may use _Auto Encoder_ like models to apply Deep Learning based _Dimensionality Reduction_.

In [ ]:
# Parameters

# Dimensionality Reduction
dataDim = 2
numOutSampleSamples = 10

# Visualization
numImg = 5

## Generate / Load Data

The notebook used [SciKit Learn's Digits Dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html).
It is similar to the MNIST dataset where the images are smaller (8 x 8) and the number of samples is 1797.

In [ ]:
# Generate / Load Data

mX, vY = load_digits(return_X_y = True)
numSamples = mX.shape[0]

print(f'The features data shape: {mX.shape}')
print(f'The labels data shape: {vY.shape}')
print(f'The unique values of the labels: {np.unique(vY)}')

In [ ]:
# Plot the Data
numImg = 3

hF, _ = plt.subplots(numImg, numImg, figsize = (4, 4))
hF = PlotMnistImages(mX, vY, numImg, tuImgSize = (8, 8), randomChoice = True, hF = hF)

## Dimensionality Reduction (Manifold Learning) by UMAP

The main parameters:

 - `n_neighbors` - Balances between local structure (Small number) and global structure (Large number).
 - `min_dist` - Density of the points in low dimension.
 - `metric` - The metric to use to evaluate the distance between samples.

In [ ]:
# Plot Dimensionality Reduction Results

def PlotDimRedResults( mX: NDArray, vL: NDArray, /, *, numNeighbors: int = 15, minDist: float = 0.1, metricType: str = 'euclidean' ) -> None:
    
    oDimRed = UMAP(n_components = dataDim, n_neighbors = numNeighbors, min_dist = minDist, metric = metricType)
    mZ = oDimRed.fit_transform(mX)

    hF, hA = plt.subplots(figsize = (6, 6))
    PlotScatterData(mZ, vY, hA = hA)

    hA.set_title(f'UMAP: numNeighbors = {numNeighbors}, minDist = {minDist}, metric = {metricType}')

In [ ]:
# Auxiliary Function

hPlotDimRedResults = lambda numNeighbors, minDist, metricType: PlotDimRedResults(mX, vY, numNeighbors = numNeighbors, minDist = minDist, metricType = metricType)

In [ ]:
# Interactive Plot
numNeighborsSlider = IntSlider(min = 2, max = 21, step = 1, value = 15, continuous_update = False, layout = Layout(width = '30%'))
minDistSlider = FloatSlider(min = 0.05, max = 1.0, step = 0.05, value = 0.1, continuous_update = False, layout = Layout(width = '30%'))
metricDropdown = Dropdown(options = ['euclidean', 'manhattan', 'cosine'], continuous_update = False, value = 'euclidean', layout = Layout(width = '30%'))
interact(hPlotDimRedResults, numNeighbors = numNeighborsSlider, minDist = minDistSlider, metricType = metricDropdown);

### Out of Sample Support  

Many of the advanced Manifold Learning methods are _Non Parameteric_ (t-SNE, UMAP).  
Hence, their ability to support _Out of Sample_ data is tricky.  

One way to overcome it is to built a _Regression Model_ to learn a parametric model of the transformation.  
Yet _UMAP_ offers a native support of Out of Sample data (So does `OpenTSNE`) wth its `transform()` method.

In [ ]:
vDataIdx, vOutOfSampleIdx = train_test_split(range(numSamples),test_size = numOutSampleSamples, random_state = seedNum, stratify = vY)